In [1]:
#Purpose: Top 10 Referral Sources for all the workflows, and how many out of those workflows were completed orders.
#Author : Abhishek Shinde
#Script Date : 12-09-2020

#Preprocessing Data

import pandas as pd
import numpy as np
import datetime
#Reading the file
df = pd.read_csv('ccs_anonymized.csv')

#Removing Index Column
df.set_index('Unnamed: 0',inplace=True)

#Removing duplicate rows
df = df.drop_duplicates(subset=['WFID'])

#Replacing '///' in Referral sources column by '/'
df['PATIENT_REFERRAL_SOURCE'] = df['PATIENT_REFERRAL_SOURCE'].str.replace('///','/')

#Creating a function to convert 5 serial number to date
def convExcelDate(inp):
    ''' 
    Takes in 5 digit serial number as input and returns date
    
            Parameters:
                inp(object): 5 digit serial number
            Returns:
                returnVal(object): date of 5 digit serial number
    '''
    inp = float(inp)
    returnVal = datetime.date(1899,12,30) + datetime.timedelta(days=inp)
    return returnVal

#Converting all date columns to proper date format
df['INTAKE_START_DATE'] = df[df['INTAKE_START_DATE'].notnull()].INTAKE_START_DATE.apply(convExcelDate)
df['INTAKE_COMPLETE_DATE'] = df[df['INTAKE_COMPLETE_DATE'].notnull()].INTAKE_COMPLETE_DATE.apply(convExcelDate)
df['SHIP_DATE'] = df[df['SHIP_DATE'].notnull()].SHIP_DATE.apply(convExcelDate)
df['ORDER_STATUS_COMPLETE_DATE'] = df[df['ORDER_STATUS_COMPLETE_DATE'].notnull()].ORDER_STATUS_COMPLETE_DATE.apply(convExcelDate)
df['WORKFLOW_CREATED_DATE'] = df[df['WORKFLOW_CREATED_DATE'].notnull()].WORKFLOW_CREATED_DATE.apply(convExcelDate)
df['WORKFLOW_START_DATE'] = df[df['WORKFLOW_START_DATE'].notnull()].WORKFLOW_START_DATE.apply(convExcelDate)
df['WORKFLOW_LAST_UPDATED'] = df[df['WORKFLOW_LAST_UPDATED'].notnull()].WORKFLOW_LAST_UPDATED.apply(convExcelDate)
df['WORKFLOW_MODIFIED_DATE'] = df[df['WORKFLOW_MODIFIED_DATE'].notnull()].WORKFLOW_MODIFIED_DATE.apply(convExcelDate)
df['WORKFLOW_CLOSED_DATE'] = df[df['WORKFLOW_CLOSED_DATE'].notnull()].WORKFLOW_CLOSED_DATE.apply(convExcelDate)

#Creating a new column which consists of month and year of when workflow was created
df['WORKFLOW_CREATED_MONTH'] = df['WORKFLOW_CREATED_DATE'].apply(lambda x: x.strftime('%m-%Y'))

#Purpose: Top 10 Referral Sources for all the workflows, and how many out of those workflows were completed orders.
#Author : Abhishek Shinde
#Script Date : 12-09-2020

#Storing top 10 Referral sources in a list
top_referral = df.PATIENT_REFERRAL_SOURCE.value_counts()[:10].index.to_list()

#Grouping Referral sources and order status column and calculating number of completed orders
top_referral_data = df[df['PATIENT_REFERRAL_SOURCE'].isin(top_referral)].groupby(['PATIENT_REFERRAL_SOURCE','ORDER_STATUS_DISPOSITION'])
top_referral_data = top_referral_data.agg({'ORDER_STATUS_DISPOSITION':'count'})
top_referral_data = top_referral_data['ORDER_STATUS_DISPOSITION']
top_referral_data = top_referral_data.unstack()

#Sorting in descending order of number of completed orders,renaming columns and storing the file in csv format
top_referral_data = top_referral_data.sort_values('completed',ascending=False)
top_referral_data.rename(columns={'completed':'Completed Orders','in process':'In Process Orders','ready':'Ready Orders'},inplace=True)
top_referral_data

ORDER_STATUS_DISPOSITION,Completed Orders,In Process Orders,Ready Orders
PATIENT_REFERRAL_SOURCE,,,
"REFERRAL (all others, not faxed) /a - Patient/Friend/Relative",44522,3145,21245
"REFERRAL (all others, not faxed) /o- OTHER Referral (detail in 3rd field)",36340,2780,16491
Referral Data Import/p - Referral from MINIMED,29887,1642,16612
"REFERRAL (all others, not faxed) /m2- Abbott- Referral from Abbott",20446,748,4959
"REFERRAL (all others, not faxed) /m2- DexCom- Referral for CGM",15566,1722,6342
Referral EMAILED IN/Dexcom_Transfer,10332,1153,7378
Referral Data Import/p - Referral from Tandem,9901,724,5547
Referral EMAILED IN/m2- DEXCOM- Referral for CGM,8160,830,5618
"REFERRAL (all others, not faxed) /m2- MiniMed-Referral from Medtronic",7619,422,3960


In [2]:
#Metric:  Top 5 products which were ordered the most. Out of those total orders, what percentage of orders are/were completed.
#Author : Abhishek Shinde
#Script Date : 12-09-2020

#Extracting top 5 products and removing 'Not Asked' From products
top_product = df[df['PRODUCT'] != 'Not Asked'].PRODUCT.value_counts()[:5].index.to_list()

#Extracting complete data of top 5 products
top_product_data = df[df['PRODUCT'].isin(top_product)]

#Calculating the percentage of order status for each product,renaming columns and saving it in csv format
top_product_order = round(pd.crosstab(index=top_product_data['PRODUCT'], columns=top_product_data['ORDER_STATUS_DISPOSITION']).stack()/len(top_product_data[top_product_data['ORDER_STATUS_DISPOSITION'].notnull()])*100,2)
top_product_order = top_product_order.unstack()
top_product_order = top_product_order.sort_values('completed',ascending = False)
top_product_order.rename(columns={'completed':'Completed Orders(%)','in process':'In Process Orders(%)','ready':'Ready Orders(%)'},inplace=True)
top_product_order

ORDER_STATUS_DISPOSITION,Completed Orders(%),In Process Orders(%),Ready Orders(%)
PRODUCT,,,
Dexcom G6,32.48,3.88,24.63
Abbott Libre,15.43,0.62,6.46
MiniMed 670G,5.25,0.04,0.91
Tandem X2 w/ Dexcom G6,4.65,0.15,1.31
Pump Supplies,2.43,0.07,1.68


In [3]:
#Metric:  M-o-M(Month on Month) trend for Top 5 Products.
#Author : Abhishek Shinde
#Script Date : 12-09-2020


#Creating a table with top 5 products and count of it each month
top_product_MoM_trend = pd.crosstab(top_product_data['PRODUCT'],top_product_data['WORKFLOW_CREATED_MONTH'])
top_product_MoM_trend.rename(columns={'01-2020':'January-2020','02-2020':'February-2020','03-2020':'March-2020','04-2020':'April-2020','05-2020':'May-2020','06-2020':'June-2020','07-2020':'July-2020','08-2019':'August-2019','09-2019':'September-2019','10-2019':'October-2019','11-2019':'November-2019','12-2019':'December-2019'},inplace=True)

#Putting all columns in correct order and storing it in csv format
top_product_MoM_trend = top_product_MoM_trend[['August-2019','September-2019','October-2019','November-2019','December-2019','January-2020','February-2020','March-2020','April-2020','May-2020','June-2020','July-2020']]
top_product_MoM_trend = top_product_MoM_trend.sort_values('August-2019',ascending = False)
top_product_MoM_trend


WORKFLOW_CREATED_MONTH,August-2019,September-2019,October-2019,November-2019,December-2019,January-2020,February-2020,March-2020,April-2020,May-2020,June-2020,July-2020
PRODUCT,,,,,,,,,,,,
Dexcom G6,512,849,885,1050,1111,1265,1363,1665,1720,2116,3168,6495
Abbott Libre,477,419,391,337,349,372,417,417,357,404,578,1138
MiniMed 670G,155,148,148,130,145,100,112,129,89,68,62,137
Tandem X2 w/ Dexcom G6,92,100,107,77,88,88,101,119,101,103,143,263
Pump Supplies,64,69,92,63,71,128,136,148,85,87,106,239


In [4]:
#Metric:  Check if the top 5 products are the same when you do a similar analysis only for the CGM workflows. 
#Author : Abhishek Shinde
#Script Date : 12-09-2020

#Selecting the data for only CGM workflows
CGM_WORKFLOW = df[df['WORKFLOW_TEMPLATE_NAME'].str.find('CGM') != -1]

#Calculating top 5 products of CGM workflows and their number of workflows
Top_Product_CGM = CGM_WORKFLOW[CGM_WORKFLOW['PRODUCT'] != 'Not Asked'].PRODUCT.value_counts()[:5].rename_axis('Product').to_frame('No. of Workflows')

#Saving it in csv format
Top_Product_CGM

,No. of Workflows
Product,
Dexcom G6,14874
Abbott Libre,5578
CGM Supplies - Dexcom,1041
CGM Supplies - Abbott,603
MiniMed Guardian 3 Link,374


In [5]:
#Metric:  Throughput trends - How my workflow days varied at an M-o-M level and what is the share of completed workflows out of all those workflows in every month. 
#Author : Abhishek Shinde
#Script Date : 12-09-2020


#Selecting columns required for analysis
throughput_trend = df[['WORKFLOW_CREATED_MONTH','WORKFLOW_DAYS_OPEN','WF_STATUS']]
#Creating a new column which indicates if workflow is completed or not
throughput_trend['WF_COMPLETED'] = throughput_trend['WF_STATUS'] == 'completed'
throughput_trend['WF_COMPLETED'] = throughput_trend['WF_COMPLETED'].astype(int)

#Grouping by each workflow created month
throughput_trend = throughput_trend.groupby('WORKFLOW_CREATED_MONTH')

#Calculating average workflow days,total workflows and number of completed workflows
throughput_trend = throughput_trend.agg({'WORKFLOW_DAYS_OPEN':'mean','WF_STATUS':'count','WF_COMPLETED':'sum'})
#Creating a new column containing percentage of completed workflows for each month
throughput_trend['WF_COMPLETED_SHARE'] = round(throughput_trend['WF_COMPLETED']/throughput_trend['WF_STATUS']*100,2)
throughput_trend['WORKFLOW_DAYS_OPEN'] = round(throughput_trend['WORKFLOW_DAYS_OPEN'],2)

#Renaming columns and storing it in csv format
throughput_trend.rename(columns={'WORKFLOW_DAYS_OPEN':'AVG_WF_DAYS_OPEN','WF_STATUS':'TOTAL_WF','WF_COMPLETED_SHARE':'WF_COMPLETED_SHARE(%)'},inplace=True)
throughput_trend

<ipython-input-5-4200fdd081cb>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  throughput_trend['WF_COMPLETED'] = throughput_trend['WF_STATUS'] == 'completed'
<ipython-input-5-4200fdd081cb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  throughput_trend['WF_COMPLETED'] = throughput_trend['WF_COMPLETED'].astype(int)


,AVG_WF_DAYS_OPEN,TOTAL_WF,WF_COMPLETED,WF_COMPLETED_SHARE(%)
WORKFLOW_CREATED_MONTH,,,,
01-2020,82.99,26569,21906,82.45
02-2020,77.81,27306,21898,80.19
03-2020,73.25,30964,24607,79.47
04-2020,68.01,32692,22876,69.97
05-2020,58.56,35942,18590,51.72
06-2020,41.56,42721,16848,39.44
07-2020,17.46,63868,2575,4.03
08-2019,108.65,24562,20858,84.92
09-2019,105.14,25139,21135,84.07


In [6]:
#Metric:  Top Patients/Payers based on the average workflow days
#Author : Abhishek Shinde
#Script Date : 12-09-2020


#Grouping by patient information and calculating average workflow days for each patient
top_patients = df.groupby(['patient_id_anony','PATIENT_AGE_GROUP']).agg({'WORKFLOW_DAYS_OPEN':'mean'})

#Sorting by average workflow days in descending order to get top patients
top_patients = top_patients.sort_values('WORKFLOW_DAYS_OPEN',ascending=False)

#Renaming columns and storing it in csv format
top_patients.rename(columns={'WORKFLOW_DAYS_OPEN':'AVG_WORKFLOW_DAYS'},inplace=True)
top_patients

,,AVG_WORKFLOW_DAYS
patient_id_anony,PATIENT_AGE_GROUP,
99122,74.3,370.0
95918,72.9,370.0
24591,57.1,370.0
100820,76.5,370.0
102382,33.5,370.0
...,...,...
112895,45.3,1.0
113486,72.2,1.0
113411,66.6,1.0
